In [ ]:
# Change path to folder with 4 images 
images_heidelberg_path = "/content/drive/MyDrive/Computer Vision Final Project/Test Heidelberg/"
# Change path to folder called final model
model_path = "/content/drive/MyDrive/Computer Vision Final Project/"

model = tf.keras.models.load_model(model_path+'final_model_image_tiles')

import numpy as np
import os 
import tensorflow as  tf
from tqdm import tqdm

files_heidelberg = os.listdir(images_heidelberg_path)

from tensorflow.keras.preprocessing.image import img_to_array,load_img

image_size = (160, 160, 3)

X_test_hd = np.zeros((len(files_heidelberg), *image_size), dtype=np.float16)

# Load the images into the NumPy array
for i, filename in tqdm(enumerate(files_heidelberg)):
    img = load_img(images_heidelberg_path+filename, target_size=image_size, color_mode="rgb")
    img_array = img_to_array(img) / 255.0
    X_test_hd[i] = img_array

def inverse_normalize_coordinates(normalized_coords):
    coords = np.zeros_like(normalized_coords)
    coords[:, 1] = (normalized_coords[:, 1]) * 90
    coords[:, 0] = (normalized_coords[:, 0]) * 180
    return coords

predictions = model.predict(X_test_hd)
predictions = inverse_normalize_coordinates(predictions)

# Show images we are predicting on  
import matplotlib.pyplot as plt

fig, axes = plt.subplots(2, 2, figsize=(8, 8))
axes = axes.ravel()

for i in range(4):
    img = X_test_hd[i]
    axes[i].imshow(img)
    axes[i].axis('off')
    axes[i].set_title(f"Image {i+1}")

plt.tight_layout()
plt.show()
print("Actual location of Heidelberg: lat:  49.3988, lng: 8.6724")
for i,pred in enumerate(predictions):
  print(f"Prediction for image {i}: lat: {pred[0]:.2f}, lng: {pred[1]:.2f}")

In [ ]:
#  if basemap is not installed run this, else skip
!pip install basemap

In [ ]:
from mpl_toolkits.basemap import Basemap
fig, ax = plt.subplots(figsize = (10,10))

map = Basemap(projection='gall',resolution = 'c',ax = ax)

# Set colors
land_color = 'white'
ocean_color = 'white'
map_color = 'lightgray'
coastline_color = 'black'

# Draw the map components
map.drawcoastlines(color=coastline_color, linewidth=0.5)
map.drawmapboundary(fill_color=ocean_color)
map.fillcontinents(color=land_color, lake_color=ocean_color)
map.drawparallels(np.arange(-90.,91.,30.),labels=[True,False,False,True])
map.drawmeridians(np.arange(-180.,181.,60.),labels=[True,False,False,True])
# Set the background color
ax = plt.gca()
ax.set_facecolor(map_color)

# Scatter plot
map.scatter(predictions[:,0],predictions[:,1], marker="x", color="red", zorder=4, s=2, latlon=True,label = "predictions")

# Add an X at the location of Heidelberg
heidelberg_lat, heidelberg_lon = 49.3988, 8.6724
x,y = map(heidelberg_lon, heidelberg_lat)
map.plot(x, y, marker='x', color='blue', markersize=7, markeredgewidth=1.5, ls = '',label ="Heidelberg")


prediction_coords = predictions[:, :2]
prediction_nums = np.arange(1, len(prediction_coords) + 1)

# Add each prediction and its number to the map
for i in range(len(prediction_coords)):
    x, y = map(prediction_coords[i, 0], prediction_coords[i, 1])
    ax.annotate(str(prediction_nums[i]), xy=(x, y), xytext=(4, 0), textcoords='offset points', fontsize=9)

# Title and legend
plt.legend(loc='lower left')
plt.title('Plot of predictions and truths, inception model, panorama images')
plt.savefig('predictions_heidelberg.png', dpi=400)
plt.show()